In [ ]:
# ===================================================================
# PHASE 6: EXCEL INTEGRATION & AUTOMATION SYSTEM
# Complete Excel reporting and application tracking
# ===================================================================
# Purpose: Generate beautiful Excel reports and track applications
# Dependencies: openpyxl, pandas, xlsxwriter
# ===================================================================

import os
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Any, Tuple
from dataclasses import dataclass, field
from pathlib import Path
import pandas as pd
import numpy as np

# ===================================================================
# LOGGING CONFIGURATION
# ===================================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("excel_automation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("ExcelAutomation")

# ===================================================================
# APPLICATION STATUS TRACKING
# ===================================================================

@dataclass
class ApplicationStatus:
    """
    Track application status for a job.
    """
    job_id: str
    company_name: str
    position_title: str
    status: str  # Not Applied, Applied, Interview, Rejected, Offer
    applied_date: Optional[str] = None
    interview_date: Optional[str] = None
    follow_up_date: Optional[str] = None
    notes: str = ""
    priority: str = "Medium"  # High, Medium, Low
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary."""
        return {
            'Job ID': self.job_id,
            'Company': self.company_name,
            'Position': self.position_title,
            'Status': self.status,
            'Applied Date': self.applied_date or '',
            'Interview Date': self.interview_date or '',
            'Follow-up Date': self.follow_up_date or '',
            'Notes': self.notes,
            'Priority': self.priority
        }


# ===================================================================
# EXCEL REPORT GENERATOR
# ===================================================================

class ExcelReportGenerator:
    """
    Generate beautiful, formatted Excel reports for job data.
    """
    
    def __init__(self):
        """Initialize Excel report generator."""
        self.logger = logging.getLogger("ExcelReportGenerator")
        
        # Try importing Excel libraries
        self.excel_engine = None
        
        # Try openpyxl first (better for formatting)
        try:
            import openpyxl
            from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
            from openpyxl.utils.dataframe import dataframe_to_rows
            from openpyxl.chart import BarChart, PieChart, Reference
            
            self.openpyxl = openpyxl
            self.Font = Font
            self.PatternFill = PatternFill
            self.Alignment = Alignment
            self.Border = Border
            self.Side = Side
            self.BarChart = BarChart
            self.PieChart = PieChart
            self.Reference = Reference
            self.dataframe_to_rows = dataframe_to_rows
            
            self.excel_engine = "openpyxl"
            self.logger.info("✅ Using openpyxl for Excel generation")
            
        except ImportError:
            self.logger.warning("⚠️ openpyxl not installed")
            self.logger.warning("Install with: pip install openpyxl")
        
        # Try xlsxwriter as alternative
        if not self.excel_engine:
            try:
                import xlsxwriter
                self.xlsxwriter = xlsxwriter
                self.excel_engine = "xlsxwriter"
                self.logger.info("✅ Using xlsxwriter for Excel generation")
            except ImportError:
                self.logger.warning("⚠️ xlsxwriter not installed")
                self.logger.warning("Install with: pip install xlsxwriter")
        
        if not self.excel_engine:
            self.logger.error("❌ No Excel library available!")
            self.logger.error("Install: pip install openpyxl")
    
    def create_job_report(
        self,
        jobs_df: pd.DataFrame,
        output_path: str = "job_search_report.xlsx",
        user_name: str = "User"
    ) -> bool:
        """
        Create comprehensive job search report.
        
        Args:
            jobs_df: DataFrame with job data
            output_path: Output Excel file path
            user_name: User's name for report header
            
        Returns:
            True if successful
        """
        if not self.excel_engine:
            self.logger.error("❌ Cannot create report - no Excel engine")
            return False
        
        self.logger.info(f"\n{'='*70}")
        self.logger.info("📊 CREATING JOB SEARCH REPORT")
        self.logger.info(f"{'='*70}\n")
        
        try:
            if self.excel_engine == "openpyxl":
                return self._create_report_openpyxl(jobs_df, output_path, user_name)
            elif self.excel_engine == "xlsxwriter":
                return self._create_report_xlsxwriter(jobs_df, output_path, user_name)
                
        except Exception as e:
            self.logger.error(f"❌ Error creating report: {e}")
            return False
    
    def _create_report_openpyxl(
        self,
        jobs_df: pd.DataFrame,
        output_path: str,
        user_name: str
    ) -> bool:
        """Create report using openpyxl."""
        
        # Create workbook
        wb = self.openpyxl.Workbook()
        wb.remove(wb.active)  # Remove default sheet
        
        # Define styles
        header_font = self.Font(bold=True, size=12, color="FFFFFF")
        header_fill = self.PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
        header_alignment = self.Alignment(horizontal="center", vertical="center")
        
        title_font = self.Font(bold=True, size=16, color="1F4E78")
        subtitle_font = self.Font(size=11, color="44546A")
        
        border_style = self.Border(
            left=self.Side(style='thin'),
            right=self.Side(style='thin'),
            top=self.Side(style='thin'),
            bottom=self.Side(style='thin')
        )
        
        # === SHEET 1: SUMMARY ===
        self.logger.info("📝 Creating Summary sheet...")
        ws_summary = wb.create_sheet("Summary", 0)
        
        # Title section
        ws_summary['B2'] = f"Job Search Report - {user_name}"
        ws_summary['B2'].font = title_font
        
        ws_summary['B3'] = f"Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}"
        ws_summary['B3'].font = subtitle_font
        
        # Key metrics
        row = 5
        ws_summary[f'B{row}'] = "KEY METRICS"
        ws_summary[f'B{row}'].font = self.Font(bold=True, size=14)
        
        row += 2
        metrics = [
            ("Total Jobs Found", len(jobs_df)),
            ("High Priority (80+)", len(jobs_df[jobs_df['final_priority_score'] >= 80])),
            ("Medium Priority (65-80)", len(jobs_df[(jobs_df['final_priority_score'] >= 65) & (jobs_df['final_priority_score'] < 80)])),
            ("Average Match Score", f"{jobs_df['final_priority_score'].mean():.1f}%"),
            ("Top Match Score", f"{jobs_df['final_priority_score'].max():.1f}%"),
            ("Unique Companies", jobs_df['company_name'].nunique()),
            ("Unique Locations", jobs_df['location_city'].nunique())
        ]
        
        for metric_name, metric_value in metrics:
            ws_summary[f'B{row}'] = metric_name
            ws_summary[f'B{row}'].font = self.Font(bold=True)
            ws_summary[f'D{row}'] = metric_value
            ws_summary[f'D{row}'].font = self.Font(size=11)
            row += 1
        
        # Top 5 companies
        row += 2
        ws_summary[f'B{row}'] = "TOP 5 HIRING COMPANIES"
        ws_summary[f'B{row}'].font = self.Font(bold=True, size=12)
        row += 1
        
        top_companies = jobs_df['company_name'].value_counts().head(5)
        for company, count in top_companies.items():
            ws_summary[f'B{row}'] = company
            ws_summary[f'D{row}'] = f"{count} positions"
            row += 1
        
        # Adjust column widths
        ws_summary.column_dimensions['B'].width = 25
        ws_summary.column_dimensions['D'].width = 20
        
        # === SHEET 2: TOP RECOMMENDATIONS ===
        self.logger.info("📝 Creating Top Recommendations sheet...")
        ws_top = wb.create_sheet("Top Recommendations", 1)
        
        # Get top 20 jobs
        top_jobs = jobs_df.nlargest(20, 'final_priority_score')
        
        # Select relevant columns
        display_columns = [
            'job_id', 'company_name', 'position_title', 'location_city',
            'work_mode', 'salary_max', 'final_priority_score',
            'skills_required', 'experience_type', 'application_deadline'
        ]
        
        top_jobs_display = top_jobs[display_columns].copy()
        
        # Format salary
        top_jobs_display['salary_max'] = top_jobs_display['salary_max'].apply(
            lambda x: f"₹{x/100000:.1f} LPA" if x > 0 else "Not specified"
        )
        
        # Rename columns
        top_jobs_display.columns = [
            'Job ID', 'Company', 'Position', 'Location', 'Work Mode',
            'Salary', 'Match Score', 'Skills Required', 'Experience', 'Deadline'
        ]
        
        # Write data
        for r_idx, row in enumerate(self.dataframe_to_rows(top_jobs_display, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                cell = ws_top.cell(row=r_idx, column=c_idx, value=value)
                
                # Header row styling
                if r_idx == 1:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = header_alignment
                    cell.border = border_style
                else:
                    cell.border = border_style
                    cell.alignment = self.Alignment(vertical="center")
        
        # Adjust column widths
        column_widths = [15, 20, 25, 15, 12, 15, 12, 40, 15, 15]
        for i, width in enumerate(column_widths, 1):
            ws_top.column_dimensions[self.openpyxl.utils.get_column_letter(i)].width = width
        
        # Freeze header row
        ws_top.freeze_panes = 'A2'
        
        # === SHEET 3: ALL JOBS ===
        self.logger.info("📝 Creating All Jobs sheet...")
        ws_all = wb.create_sheet("All Jobs", 2)
        
        # Write all jobs
        all_jobs_display = jobs_df[display_columns].copy()
        all_jobs_display['salary_max'] = all_jobs_display['salary_max'].apply(
            lambda x: f"₹{x/100000:.1f} LPA" if x > 0 else "Not specified"
        )
        all_jobs_display.columns = [
            'Job ID', 'Company', 'Position', 'Location', 'Work Mode',
            'Salary', 'Match Score', 'Skills Required', 'Experience', 'Deadline'
        ]
        
        for r_idx, row in enumerate(self.dataframe_to_rows(all_jobs_display, index=False, header=True), 1):
            for c_idx, value in enumerate(row, 1):
                cell = ws_all.cell(row=r_idx, column=c_idx, value=value)
                
                if r_idx == 1:
                    cell.font = header_font
                    cell.fill = header_fill
                    cell.alignment = header_alignment
                    cell.border = border_style
                else:
                    cell.border = border_style
        
        # Adjust column widths
        for i, width in enumerate(column_widths, 1):
            ws_all.column_dimensions[self.openpyxl.utils.get_column_letter(i)].width = width
        
        ws_all.freeze_panes = 'A2'
        
        # === SHEET 4: BY LOCATION ===
        self.logger.info("📝 Creating By Location sheet...")
        ws_location = wb.create_sheet("By Location", 3)
        
        location_summary = jobs_df.groupby('location_city').agg({
            'job_id': 'count',
            'final_priority_score': 'mean',
            'salary_max': 'mean'
        }).round(2)
        
        location_summary.columns = ['Job Count', 'Avg Match Score', 'Avg Salary']
        location_summary = location_summary.sort_values('Job Count', ascending=False)
        location_summary['Avg Salary'] = location_summary['Avg Salary'].apply(
            lambda x: f"₹{x/100000:.1f} LPA" if x > 0 else "N/A"
        )
        
        # Write location data
        ws_location['A1'] = "Location"
        ws_location['B1'] = "Job Count"
        ws_location['C1'] = "Avg Match Score"
        ws_location['D1'] = "Avg Salary"
        
        for cell in ['A1', 'B1', 'C1', 'D1']:
            ws_location[cell].font = header_font
            ws_location[cell].fill = header_fill
            ws_location[cell].alignment = header_alignment
        
        for r_idx, (location, row) in enumerate(location_summary.iterrows(), 2):
            ws_location[f'A{r_idx}'] = location
            ws_location[f'B{r_idx}'] = row['Job Count']
            ws_location[f'C{r_idx}'] = row['Avg Match Score']
            ws_location[f'D{r_idx}'] = row['Avg Salary']
        
        ws_location.column_dimensions['A'].width = 20
        ws_location.column_dimensions['B'].width = 12
        ws_location.column_dimensions['C'].width = 18
        ws_location.column_dimensions['D'].width = 15
        
        # === SHEET 5: STATISTICS ===
        self.logger.info("📝 Creating Statistics sheet...")
        ws_stats = wb.create_sheet("Statistics", 4)
        
        row = 1
        ws_stats[f'A{row}'] = "SALARY STATISTICS"
        ws_stats[f'A{row}'].font = self.Font(bold=True, size=14)
        
        row += 2
        salary_data = jobs_df[jobs_df['salary_max'] > 0]['salary_max']
        
        salary_stats = [
            ("Jobs with Salary Info", len(salary_data)),
            ("Average Salary", f"₹{salary_data.mean()/100000:.2f} LPA"),
            ("Median Salary", f"₹{salary_data.median()/100000:.2f} LPA"),
            ("Minimum Salary", f"₹{salary_data.min()/100000:.2f} LPA"),
            ("Maximum Salary", f"₹{salary_data.max()/100000:.2f} LPA")
        ]
        
        for stat_name, stat_value in salary_stats:
            ws_stats[f'A{row}'] = stat_name
            ws_stats[f'A{row}'].font = self.Font(bold=True)
            ws_stats[f'C{row}'] = stat_value
            row += 1
        
        row += 2
        ws_stats[f'A{row}'] = "WORK MODE DISTRIBUTION"
        ws_stats[f'A{row}'].font = self.Font(bold=True, size=14)
        
        row += 2
        work_mode_dist = jobs_df['work_mode'].value_counts()
        for mode, count in work_mode_dist.items():
            percentage = (count / len(jobs_df)) * 100
            ws_stats[f'A{row}'] = mode
            ws_stats[f'A{row}'].font = self.Font(bold=True)
            ws_stats[f'C{row}'] = f"{count} ({percentage:.1f}%)"
            row += 1
        
        ws_stats.column_dimensions['A'].width = 25
        ws_stats.column_dimensions['C'].width = 20
        
        # Save workbook
        wb.save(output_path)
        
        self.logger.info(f"✅ Report saved: {output_path}")
        self.logger.info(f"📄 Sheets created: {len(wb.sheetnames)}")
        self.logger.info(f"{'='*70}\n")
        
        return True
    
    def _create_report_xlsxwriter(
        self,
        jobs_df: pd.DataFrame,
        output_path: str,
        user_name: str
    ) -> bool:
        """Create report using xlsxwriter (simplified version)."""
        
        # Create workbook
        workbook = self.xlsxwriter.Workbook(output_path)
        
        # Define formats
        header_format = workbook.add_format({
            'bold': True,
            'bg_color': '#4472C4',
            'font_color': 'white',
            'align': 'center',
            'valign': 'vcenter',
            'border': 1
        })
        
        # Summary sheet
        ws_summary = workbook.add_worksheet('Summary')
        ws_summary.write('B2', f"Job Search Report - {user_name}")
        
        # Top recommendations sheet
        ws_top = workbook.add_worksheet('Top Recommendations')
        top_jobs = jobs_df.nlargest(20, 'final_priority_score')
        
        # Write headers
        headers = ['Company', 'Position', 'Location', 'Salary', 'Match Score']
        for col, header in enumerate(headers):
            ws_top.write(0, col, header, header_format)
        
        # Write data
        for row_idx, (_, job) in enumerate(top_jobs.iterrows(), 1):
            ws_top.write(row_idx, 0, job['company_name'])
            ws_top.write(row_idx, 1, job['position_title'])
            ws_top.write(row_idx, 2, job['location_city'])
            salary = f"₹{job['salary_max']/100000:.1f} LPA" if job['salary_max'] > 0 else "N/A"
            ws_top.write(row_idx, 3, salary)
            ws_top.write(row_idx, 4, f"{job['final_priority_score']:.1f}")
        
        workbook.close()
        
        self.logger.info(f"✅ Report saved: {output_path}")
        return True


# ===================================================================
# APPLICATION TRACKER
# ===================================================================

class ApplicationTracker:
    """
    Track job applications with status updates.
    """
    
    def __init__(self, tracker_file: str = "application_tracker.xlsx"):
        """
        Initialize application tracker.
        
        Args:
            tracker_file: Excel file to store tracking data
        """
        self.logger = logging.getLogger("ApplicationTracker")
        self.tracker_file = tracker_file
        self.applications: Dict[str, ApplicationStatus] = {}
        
        # Load existing tracker
        self.load_tracker()
    
    def load_tracker(self):
        """Load existing tracker data."""
        if not os.path.exists(self.tracker_file):
            self.logger.info("📝 No existing tracker found - creating new")
            return
        
        try:
            df = pd.read_excel(self.tracker_file, sheet_name='Applications')
            
            for _, row in df.iterrows():
                app_status = ApplicationStatus(
                    job_id=row['Job ID'],
                    company_name=row['Company'],
                    position_title=row['Position'],
                    status=row['Status'],
                    applied_date=row.get('Applied Date'),
                    interview_date=row.get('Interview Date'),
                    follow_up_date=row.get('Follow-up Date'),
                    notes=row.get('Notes', ''),
                    priority=row.get('Priority', 'Medium')
                )
                self.applications[app_status.job_id] = app_status
            
            self.logger.info(f"✅ Loaded {len(self.applications)} tracked applications")
            
        except Exception as e:
            self.logger.error(f"❌ Error loading tracker: {e}")
    
    def add_application(
        self,
        job_id: str,
        company_name: str,
        position_title: str,
        status: str = "Not Applied",
        priority: str = "Medium"
    ):
        """Add new application to track."""
        app_status = ApplicationStatus(
            job_id=job_id,
            company_name=company_name,
            position_title=position_title,
            status=status,
            priority=priority
        )
        
        self.applications[job_id] = app_status
        self.logger.info(f"✅ Added application: {company_name} - {position_title}")
    
    def update_status(
        self,
        job_id: str,
        new_status: str,
        notes: Optional[str] = None
    ):
        """Update application status."""
        if job_id not in self.applications:
            self.logger.error(f"❌ Job {job_id} not found in tracker")
            return
        
        app = self.applications[job_id]
        old_status = app.status
        app.status = new_status
        
        # Update dates based on status
        if new_status == "Applied" and not app.applied_date:
            app.applied_date = datetime.now().strftime('%Y-%m-%d')
        elif new_status == "Interview" and not app.interview_date:
            app.interview_date = datetime.now().strftime('%Y-%m-%d')
        
        if notes:
            app.notes = notes
        
        self.logger.info(f"✅ Updated {job_id}: {old_status} → {new_status}")
    
    def save_tracker(self):
        """Save tracker to Excel."""
        try:
            # Convert to DataFrame
            data = [app.to_dict() for app in self.applications.values()]
            df = pd.DataFrame(data)
            
            # Sort by priority and status
            priority_order = {'High': 0, 'Medium': 1, 'Low': 2}
            df['priority_rank'] = df['Priority'].map(priority_order)
            df = df.sort_values(['priority_rank', 'Status'])
            df = df.drop('priority_rank', axis=1)
            
            # Save to Excel
            with pd.ExcelWriter(self.tracker_file, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='Applications', index=False)
                
                # Format the sheet
                workbook = writer.book
                worksheet = writer.sheets['Applications']
                
                # Auto-fit columns
                for column in worksheet.columns:
                    max_length = 0
                    column = [cell for cell in column]
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(cell.value)
                        except:
                            pass
                    adjusted_width = min(max_length + 2, 50)
                    worksheet.column_dimensions[column[0].column_letter].width = adjusted_width
            
            self.logger.info(f"💾 Tracker saved: {self.tracker_file}")
            
        except Exception as e:
            self.logger.error(f"❌ Error saving tracker: {e}")
    
    def get_summary(self) -> Dict[str, int]:
        """Get summary statistics."""
        summary = {
            'total': len(self.applications),
            'not_applied': 0,
            'applied': 0,
            'interview': 0,
            'rejected': 0,
            'offer': 0
        }
        
        for app in self.applications.values():
            status_lower = app.status.lower()
            if 'not applied' in status_lower:
                summary['not_applied'] += 1
            elif 'applied' in status_lower:
                summary['applied'] += 1
            elif 'interview' in status_lower:
                summary['interview'] += 1
            elif 'reject' in status_lower:
                summary['rejected'] += 1
            elif 'offer' in status_lower:
                summary['offer'] += 1
        
        return summary


# ===================================================================
# EXCEL AUTOMATION ORCHESTRATOR
# ===================================================================

class ExcelAutomation:
    """
    Complete Excel automation system.
    Combines reporting and tracking.
    """
    
    def __init__(self):
        """Initialize Excel automation."""
        self.logger = logging.getLogger("ExcelAutomation")
        self.report_generator = ExcelReportGenerator()
        self.tracker = ApplicationTracker()
        
        self.logger.info("="*70)
        self.logger.info("📊 EXCEL AUTOMATION INITIALIZED")
        self.logger.info("="*70 + "\n")
    
    def create_complete_package(
        self,
        jobs_df: pd.DataFrame,
        user_name: str = "User",
        output_dir: str = "./reports"
    ) -> Dict[str, str]:
        """
        Create complete Excel package with all reports.
        
        Args:
            jobs_df: Jobs DataFrame
            user_name: User's name
            output_dir: Output directory
            
        Returns:
            Dictionary with file paths
        """
        self.logger.info(f"\n{'='*70}")
        self.logger.info("📦 CREATING COMPLETE EXCEL PACKAGE")
        self.logger.info(f"{'='*70}\n")
        
        # Create output directory
        Path(output_dir).mkdir(exist_ok=True)
        
        files_created = {}
        
        # 1. Main job report
        report_path = os.path.join(output_dir, f"job_search_report_{datetime.now().strftime('%Y%m%d')}.xlsx")
        success = self.report_generator.create_job_report(jobs_df, report_path, user_name)
        
        if success:
            files_created['main_report'] = report_path
            self.logger.info(f"✅ Main report: {report_path}")
        
        # 2. Application tracker template
        tracker_path = os.path.join(output_dir, "application_tracker.xlsx")
        self._create_tracker_template(jobs_df, tracker_path)
        files_created['tracker'] = tracker_path
        self.logger.info(f"✅ Tracker: {tracker_path}")
        
        self.logger.info(f"\n{'='*70}")
        self.logger.info(f"✅ PACKAGE COMPLETE - {len(files_created)} files")
        self.logger.info(f"{'='*70}\n")
        
        return files_created
    
    def _create_tracker_template(self, jobs_df: pd.DataFrame, output_path: str):
        """Create application tracker template."""
        # Get top 50 jobs
        top_jobs = jobs_df.nlargest(50, 'final_priority_score')
        
        # Prepare tracker data
        tracker_data = []
        for _, job in top_jobs.iterrows():
            tracker_data.append({
                'Job ID': job['job_id'],
                'Company': job['company_name'],
                'Position': job['position_title'],
                'Status': 'Not Applied',
                'Applied Date': '',
                'Interview Date': '',
                'Follow-up Date': '',
                'Notes': '',
                'Priority': 'Medium'
            })
        
        df = pd.DataFrame(tracker_data)
        df.to_excel(output_path, sheet_name='Applications', index=False)


# ===================================================================
# TESTING & EXAMPLES
# ===================================================================

def test_excel_system():
    """Test Excel automation system."""
    logger.info("\n" + "="*70)
    logger.info("🧪 TESTING EXCEL SYSTEM")
    logger.info("="*70 + "\n")
    
    # Load sample data
    try:
        csv_path = r"D:\Projects By Month\November 2025\Placement Mail Analysis System\.venv\Phase_scripts\Phase 4\prioritized_jobs.csv"
        jobs_df = pd.read_csv(csv_path)
        logger.info(f"✅ Loaded {len(jobs_df)} jobs")
    except Exception as e:
        logger.error(f"❌ Failed to load jobs: {e}")
        return
    
    # Initialize system
    excel_system = ExcelAutomation()
    
    # Create complete package
    files = excel_system.create_complete_package(
        jobs_df=jobs_df,
        user_name="Test User",
        output_dir="./excel_reports"
    )
    
    logger.info(f"\n📁 Files created:")
    for file_type, file_path in files.items():
        logger.info(f"  - {file_type}: {file_path}")


# ===================================================================
# MAIN EXECUTION
# ===================================================================

if __name__ == "__main__":
    test_excel_system()